# GAI Project 3 PEFT on GLUE benchmarks

In [1]:
# 若沒有安裝 transformers 和 datasets 套件，請取消以下註解並執行
!pip install transformers==4.38.0
!pip install datasets
!pip install torch
!pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 34.9 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.2
    Uninstalling transformers-4.40.2:
      Successfully uninstalled transformers-4.40.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 10.1 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingf

In [2]:
import transformers
import peft
import torch
import datasets
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, TaskType
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

In [ ]:
print("PyTorch 的版本為: {}".format(torch.__version__))
print("Hugging Face Transformers 的版本為: {}".format(transformers.__version__))
print("Hugging Face Datasets 的版本為: {}".format(datasets.__version__))
print("PEFT 的版本為: {}".format(peft.__version__))

PyTorch 的版本為: 2.2.1+cu121
Hugging Face Transformers 的版本為: 4.38.0
Hugging Face Datasets 的版本為: 2.19.1
PEFT 的版本為: 0.10.0


## Dataset 1: **MRPC**

In [3]:
# 載入資料集
dataset = load_dataset("glue", "mrpc")

# 初始化 Tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [4]:
# 資料前處理函數
def preprocess_function(examples):
    return tokenizer(examples['sentence1'], examples['sentence2'], truncation=True, padding="max_length", max_length=128)

# 對資料進行前處理
dataset = dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [ ]:
# 載入模型
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model2 = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# 計算模型評價指標的函數
def compute_metrics_mrpc(eval_pred):
    import numpy as np
    from sklearn.metrics import accuracy_score, f1_score

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions)
    return {'accuracy': acc, 'f1': f1}

### BitFit


In [ ]:
# 修改模型的訓練參數配置，僅對偏差進行訓練
for param in model.parameters():
    param.requires_grad = False
for name, param in model.named_parameters():
    if 'bias' in name:
        param.requires_grad = True

In [ ]:
# 設定訓練超參數
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=4,
    learning_rate=5e-4,
    weight_decay=0.01,
    evaluation_strategy="steps",
    logging_dir='./logs',
    logging_steps=10
)

In [ ]:
# 初始化Trainer並訓練模型
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['validation'],
    compute_metrics=compute_metrics_mrpc
)

# 開始訓練
trainer.train()

# 評估模型
trainer.evaluate()


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,F1
10,0.631800,0.605744,0.683824,0.812227
20,0.631600,0.597526,0.683824,0.812227
30,0.582000,0.596443,0.683824,0.812227
40,0.587700,0.590521,0.700980,0.818452
50,0.593900,0.577774,0.696078,0.817109
60,0.579500,0.571819,0.698529,0.816143
70,0.579700,0.561122,0.700980,0.817910
80,0.570800,0.558151,0.698529,0.813354
90,0.581400,0.552160,0.698529,0.811639
100,0.559200,0.544831,0.713235,0.824060


{'eval_loss': 0.45934438705444336,
 'eval_accuracy': 0.8088235294117647,
 'eval_f1': 0.8682432432432433,
 'eval_runtime': 3.246,
 'eval_samples_per_second': 125.692,
 'eval_steps_per_second': 8.01,
 'epoch': 9.91}

### LoRA

In [ ]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    target_modules='all-linear',
    task_type=TaskType.SEQ_CLS
)
model = get_peft_model(model2, peft_config)
model.print_trainable_parameters()

trainable params: 2,692,642 || all params: 112,188,740 || trainable%: 2.40010004569086


In [ ]:
# 設定訓練超參數
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=4,
    learning_rate=5e-4,
    weight_decay=0.01,
    evaluation_strategy="steps",
    logging_dir='./logs',
    logging_steps=10
)

In [ ]:
# 初始化Trainer並訓練模型
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['validation'],
    compute_metrics=compute_metrics_mrpc
)

# 開始訓練
trainer.train()

# 評估模型
trainer.evaluate()


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,F1
10,0.651000,0.583734,0.718137,0.823890
20,0.607100,0.541391,0.742647,0.837713
30,0.553700,0.533302,0.742647,0.840183
40,0.533000,0.614518,0.727941,0.833583
50,0.537400,0.498606,0.767157,0.850394
60,0.465500,0.392586,0.840686,0.890388
70,0.404800,0.424437,0.806373,0.872786
80,0.354200,0.361808,0.852941,0.890909
90,0.390200,0.375764,0.843137,0.880150
100,0.371800,0.348210,0.862745,0.904437


Checkpoint destination directory ./results/checkpoint-500 already exists and is non-empty. Saving will proceed but saved results may be invalid.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


{'eval_loss': 0.7245796322822571,
 'eval_accuracy': 0.8553921568627451,
 'eval_f1': 0.8977469670710572,
 'eval_runtime': 3.8487,
 'eval_samples_per_second': 106.011,
 'eval_steps_per_second': 6.756,
 'epoch': 9.91}

### 以MRPC為例，測試不同LoRA rank(r)對於結果的影響

In [6]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model2 = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### rank = 32

In [7]:
peft_config = LoraConfig(
    r=32,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    target_modules='all-linear',
    task_type=TaskType.SEQ_CLS
)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 5,383,746 || all params: 114,892,164 || trainable%: 4.6859122611703965


In [8]:
# 設定訓練超參數
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=4,
    learning_rate=5e-4,
    weight_decay=0.01,
    evaluation_strategy="steps",
    logging_dir='./logs',
    logging_steps=10
)

In [9]:
# 初始化Trainer並訓練模型
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['validation'],
    compute_metrics=compute_metrics_mrpc
)

# 開始訓練
trainer.train()

# 評估模型
trainer.evaluate()


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,F1
10,0.672700,0.605036,0.705882,0.821429
20,0.608100,0.543345,0.740196,0.826797
30,0.528500,0.519556,0.747549,0.810313
40,0.521100,0.531448,0.764706,0.850932
50,0.506700,0.411603,0.825980,0.879457
60,0.491700,0.378976,0.828431,0.872727
70,0.448700,0.391844,0.833333,0.887417
80,0.381100,0.337117,0.857843,0.897527
90,0.351900,0.341748,0.872549,0.909722
100,0.344800,0.346025,0.835784,0.887772


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


{'eval_loss': 0.6160754561424255,
 'eval_accuracy': 0.8602941176470589,
 'eval_f1': 0.900523560209424,
 'eval_runtime': 3.708,
 'eval_samples_per_second': 110.033,
 'eval_steps_per_second': 7.012,
 'epoch': 9.91}

#### rank = 8

In [10]:
peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    target_modules='all-linear',
    task_type=TaskType.SEQ_CLS
)
model = get_peft_model(model2, peft_config)
model.print_trainable_parameters()

trainable params: 1,347,090 || all params: 110,837,028 || trainable%: 1.2153790338008703


In [11]:
# 設定訓練超參數
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=4,
    learning_rate=5e-4,
    weight_decay=0.01,
    evaluation_strategy="steps",
    logging_dir='./logs',
    logging_steps=10
)

In [12]:
# 初始化Trainer並訓練模型
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['validation'],
    compute_metrics=compute_metrics_mrpc
)

# 開始訓練
trainer.train()

# 評估模型
trainer.evaluate()


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,F1
10,0.764400,0.659187,0.683824,0.812227
20,0.670400,0.627630,0.683824,0.812227
30,0.595700,0.632863,0.693627,0.815362
40,0.571000,0.574341,0.737745,0.835131
50,0.547800,0.531255,0.752451,0.842924
60,0.532500,0.492612,0.784314,0.861635
70,0.456000,0.445539,0.823529,0.877551
80,0.413600,0.455459,0.816176,0.874791
90,0.438700,0.429836,0.808824,0.860215
100,0.457700,0.401039,0.825980,0.864762


Checkpoint destination directory ./results/checkpoint-500 already exists and is non-empty. Saving will proceed but saved results may be invalid.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


{'eval_loss': 0.8137052655220032,
 'eval_accuracy': 0.8578431372549019,
 'eval_f1': 0.9003436426116839,
 'eval_runtime': 3.6489,
 'eval_samples_per_second': 111.816,
 'eval_steps_per_second': 7.125,
 'epoch': 9.91}

## Dataset 2: **SST-2**

In [ ]:
# 載入資料集
dataset_sst2 = load_dataset("glue", "sst2")

# 初始化 Tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
# 資料前處理函數
def preprocess_sst2(examples):
    return tokenizer(examples['sentence'], truncation=True, padding="max_length", max_length=128)

# 對資料進行前處理
dataset_sst2 = dataset_sst2.map(preprocess_sst2, batched=True)

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [ ]:
# 載入模型
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model2 = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# 計算模型評價指標的函數
def compute_metrics_sst2(eval_pred):
    import numpy as np
    from sklearn.metrics import accuracy_score

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, predictions)
    return {'accuracy': acc}

### BitFit

In [ ]:
# 修改模型的訓練參數配置，僅對偏差進行訓練
for param in model.parameters():
    param.requires_grad = False
for name, param in model.named_parameters():
    if 'bias' in name:
        param.requires_grad = True

In [ ]:
# 設定訓練超參數
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=4,
    learning_rate=5e-3,
    weight_decay=0.01,
    evaluation_strategy="steps",
    logging_dir='./logs',
    logging_steps=50
)

In [ ]:
# 初始化Trainer並訓練模型
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_sst2['train'],
    eval_dataset=dataset_sst2['validation'],
    compute_metrics=compute_metrics_sst2
)

# 開始訓練
trainer.train()

# 評估模型
trainer.evaluate()


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
50,0.525100,0.370759,0.846330
100,0.393000,0.506420,0.792431
150,0.373600,0.301862,0.872706
200,0.322900,0.340545,0.852064
250,0.321600,0.288362,0.880734
300,0.350800,0.317135,0.869266
350,0.306700,0.384499,0.849771
400,0.282100,0.295219,0.885321
450,0.294700,0.268168,0.889908
500,0.292200,0.317729,0.873853


Checkpoint destination directory ./results/checkpoint-500 already exists and is non-empty. Saving will proceed but saved results may be invalid.


{'eval_loss': 0.22979994118213654,
 'eval_accuracy': 0.9139908256880734,
 'eval_runtime': 6.9454,
 'eval_samples_per_second': 125.55,
 'eval_steps_per_second': 7.919,
 'epoch': 1.0}

### LoRA

In [ ]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    target_modules='all-linear',
    task_type=TaskType.SEQ_CLS
)
model = get_peft_model(model2, peft_config)
model.print_trainable_parameters()

trainable params: 2,692,642 || all params: 112,188,740 || trainable%: 2.40010004569086


In [ ]:
# 設定訓練超參數
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=4,
    learning_rate=5e-4,
    weight_decay=0.01,
    evaluation_strategy="steps",
    logging_dir='./logs',
    logging_steps=50
)

In [ ]:
# 初始化Trainer並訓練模型
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_sst2['train'],
    eval_dataset=dataset_sst2['validation'],
    compute_metrics=compute_metrics_sst2
)

# 開始訓練
trainer.train()

# 評估模型
trainer.evaluate()


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
50,0.472800,0.295904,0.881881
100,0.298000,0.309265,0.880734
150,0.299100,0.343382,0.858945
200,0.258500,0.240382,0.912844
250,0.264600,0.232583,0.911697
300,0.221200,0.241471,0.896789
350,0.219000,0.321191,0.878440
400,0.220000,0.234878,0.915138
450,0.211600,0.239746,0.911697
500,0.208300,0.245164,0.904817


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


{'eval_loss': 0.2146022468805313,
 'eval_accuracy': 0.9162844036697247,
 'eval_runtime': 8.2494,
 'eval_samples_per_second': 105.704,
 'eval_steps_per_second': 6.667,
 'epoch': 1.0}

### 以SST2為例，測試不同LoRA rank(r)對於結果的影響

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model2 = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### rank = 32

In [ ]:
peft_config = LoraConfig(
    r=32,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    target_modules='all-linear',
    task_type=TaskType.SEQ_CLS
)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 5,383,746 || all params: 114,892,164 || trainable%: 4.6859122611703965


In [ ]:
# 設定訓練超參數
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=4,
    learning_rate=5e-4,
    weight_decay=0.01,
    evaluation_strategy="steps",
    logging_dir='./logs',
    logging_steps=50
)

In [ ]:
# 初始化Trainer並訓練模型
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_sst2['train'],
    eval_dataset=dataset_sst2['validation'],
    compute_metrics=compute_metrics_sst2
)

# 開始訓練
trainer.train()

# 評估模型
trainer.evaluate()


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
50,0.461100,0.296407,0.880734
100,0.299500,0.274411,0.894495
150,0.298600,0.251802,0.895642
200,0.262000,0.245589,0.904817
250,0.267200,0.223816,0.912844
300,0.220400,0.233739,0.907110
350,0.221600,0.263727,0.899083
400,0.213300,0.207231,0.925459
450,0.222000,0.223124,0.916284
500,0.214300,0.221145,0.911697


Checkpoint destination directory ./results/checkpoint-500 already exists and is non-empty. Saving will proceed but saved results may be invalid.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Checkpoint destination directory ./results/checkpoint-1000 already exists and is non-empty. Saving will proceed but saved results may be invalid.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


{'eval_loss': 0.2056942582130432,
 'eval_accuracy': 0.9254587155963303,
 'eval_runtime': 8.3619,
 'eval_samples_per_second': 104.283,
 'eval_steps_per_second': 6.577,
 'epoch': 1.0}

#### rank = 8

In [ ]:
peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    target_modules='all-linear',
    task_type=TaskType.SEQ_CLS
)
model = get_peft_model(model2, peft_config)
model.print_trainable_parameters()

trainable params: 1,347,090 || all params: 110,837,028 || trainable%: 1.2153790338008703


In [ ]:
# 設定訓練超參數
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=4,
    learning_rate=5e-4,
    weight_decay=0.01,
    evaluation_strategy="steps",
    logging_dir='./logs',
    logging_steps=50
)

In [ ]:
# 初始化Trainer並訓練模型
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_sst2['train'],
    eval_dataset=dataset_sst2['validation'],
    compute_metrics=compute_metrics_sst2
)

# 開始訓練
trainer.train()

# 評估模型
trainer.evaluate()


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
50,0.470300,0.331177,0.866972
100,0.316200,0.265497,0.902523
150,0.301700,0.381497,0.857798
200,0.268800,0.237999,0.905963
250,0.258600,0.224508,0.913991
300,0.216100,0.222196,0.908257
350,0.229000,0.281925,0.888761
400,0.221000,0.230207,0.910550


Step,Training Loss,Validation Loss,Accuracy
50,0.470300,0.331177,0.866972
100,0.316200,0.265497,0.902523
150,0.301700,0.381497,0.857798
200,0.268800,0.237999,0.905963
250,0.258600,0.224508,0.913991
300,0.216100,0.222196,0.908257
350,0.229000,0.281925,0.888761
400,0.221000,0.230207,0.910550
450,0.216400,0.223828,0.912844
500,0.217100,0.217038,0.908257


Checkpoint destination directory ./results/checkpoint-500 already exists and is non-empty. Saving will proceed but saved results may be invalid.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Checkpoint destination directory ./results/checkpoint-1000 already exists and is non-empty. Saving will proceed but saved results may be invalid.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


{'eval_loss': 0.21130798757076263,
 'eval_accuracy': 0.9174311926605505,
 'eval_runtime': 8.215,
 'eval_samples_per_second': 106.147,
 'eval_steps_per_second': 6.695,
 'epoch': 1.0}